# Solving a mincost SAT problem using QAOA

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

We want to find an (almost) optimal configuration in an attributed feature model.

The system can be described by the following boolean formulae:
$
(x_1 \vee x_2) \wedge (x_3 \oplus x_4) \wedge (x_5 \implies x_6)
\iff
(x_1 \vee x_2) \wedge (\neg x_3 \vee \neg x_4) \wedge (x_3 \vee x_4) \wedge (\neg x_5 \vee x_6)
$

The right hand side is in conjunctive normal form (CNF).

Furthermore, there is an implementation cost associated with each feature, as shown in the table below.

| Feature | Cost $c_i$ |
| --- | --- |
| $x_1$ | 30 |
| $x_2$ | 20 |
| $x_3$ | 25 |
| $x_4$ | 50 |
| $x_5$ | 10 |
| $x_6$ | 10 |


# Quantum Approximation Optimization Algorithm
Approximates the solution of an combinatorial optimization problem consisting of:
- $n$ binary variables
- $m$ clauses
- objective function $C(\vec z)$

The domain of the problem is unconstrained, thus the algorithms goal is to find an (almost) optimal bistring $\vec z=z_1...z_n$

It refines VQA and uses the Alternating Operator Ansatz.
The algorithm consists of a classical and quantum part.

On a quantum computer a circuit is constructed which is parameterized by $\vec \gamma$ and $\vec \beta$.
Initially the uniform superposition state $H^{\otimes n}$ is prepared.

Two operators $U_C$ and $U_M$ are constructed and parametrised with the parameters $\vec \gamma$ and $\vec \beta$ respectively.
The phase-separating operator $U_C$ encodes $C$  and applies a phase shift $e^{-i \vec \gamma}$ on every computational basis state for every clause that is fulfilled.
The mixing operator $U_M$ changes the amplitude of solutions using rotation $R_x$.

Both $U_C$  and $U_M$ are then applied $p$ times according to the hyper-parameter $p \in \mathbf{N}$.
Finally measurements gates are added.

The circuit has a shallow circuit depth of at most $mp+m$.

On a classical computer the cost of $C(\vec z)$ for the current evaluation is evaluated.
Either the process is terminated if the termination condition is met ($C(\vec z)$ is sufficiently low), or the parameters $\vec \gamma$ and $\vec \beta$ are optimized classically.

$-2\pi\lt\gamma\lt2\pi$ and $-\pi\lt\beta\lt\pi$


## Quantum circuit
We start by defining the parameteriezed circuit.

In [ ]:
# Notebook Setup
from IPython.core import page
page.page = print

# Imports used for examples
from qiskit.visualization import plot_histogram
from pprint import pprint

### Initialization
Uniform superposition by applying Hadamard gates $H$ on every qubit.

In [ ]:
# Uniform Superposition Initialization
from configproblem.fragments.quantum_states import superposition_circuit, add_all_hadamards
%psource superposition_circuit
%psource add_all_hadamards

In [ ]:
superposition_circuit(2).draw(output="mpl")

### Mixing operator
The mixing operator $U_M$ applies a rotation around $X$ of $2*\beta$ on every qubit using $R_x$ gates.

In [ ]:
# Mixer Hamiltonian
import configproblem.qaoa.qaoa_mixer as mixer
%psource mixer.standard_mixer

In [ ]:
from qiskit.circuit import Parameter

example_qc_mixer = mixer.standard_mixer(2, Parameter("$\\beta$"))
example_qc_mixer.draw(output="mpl")

### Phase-separating operator
The phase-separating operator $U_C$ encodes $C$ and can be derived from a cost Hamiltonian $H_C$ in Ising-form. 
We can describe $H_C$ in a form where for $m$ clauses operating on one or two qubits (because the domain is unconstrained) the values of $C$ are encoded.
We later describe how such a Hamiltonian can be constructed for our concrete problem class.

In [ ]:
# Hamiltonians may be described as a dict of tuples describing acting qubits and a value for each clause
# hamiltonian = {(q1?, q2?, ...) : factor}
sat_hamiltonian = {
    () : 1,
    (0,): 0.25,
    (1,): 0.25,
    (4,): -0.25,
    (5,): 0.25,
    (0, 1): 0.25,
    (2, 3): 0.5,
    (4, 5): -0.25
}

Clauses on one qubits are then translated into $R_z$ rotations and clauses on two qubits are translated into the symmetric $R_{zz}$ gate.

In [ ]:
# Cost Hamiltonian
from qaoa_mincost_sat import problem_circuit
%psource problem_circuit

In [ ]:
example_qc_problem = problem_circuit(sat_hamiltonian, 6, Parameter("$\\gamma$"))
example_qc_problem.draw(output="mpl")

### QAOA circuit
Now we can create a QAOA circuit for a problem hamiltonian.

The circuit can also be warmstarted by initializing a specific state $\ket{s}$ instead of a uniform superposition.

In [ ]:
# QAOA Circuit
from qaoa_application import qaoa_circuit
%psource qaoa_circuit

In [ ]:
example_qaoa_circuit, _, _ = qaoa_circuit(problem_circuit, mixer.standard_mixer, sat_hamiltonian, 6, 1)
example_qaoa_circuit.draw(output="mpl")

### Quantum routine
Executes QAOA circuit and returns circuit and results

In [ ]:
# QAOA Quantum Procedure
from qaoa_application import quantum
%psource quantum

In [ ]:
counts, qc = quantum(problem_circuit, mixer.standard_mixer, sat_hamiltonian, 6, 1, [1], [1])
qc.draw(output="mpl")

In [ ]:
plot_histogram(counts, figsize=(40, 10))

## Creating a problem specific Hamiltonian

### Creating a cost function satisfying features

[Glover](https://arxiv.org/abs/1811.11538) described quadratic penalties for the 2-SAT problem by treating inputs as $0/1$ values, forming a traditional constraint and then deriving a quadratic penalty. We summarize their work here:

| Clause Type | Example | Constraint                   | Penalty |
|---|---|------------------------------|---|
| No Negations  | $$(x_i \vee x_j)$$               | $$x_i + x_j \geq 1$$         | $$(1 - x_i - x_j + x_i x_j)$$
| One Negation  | $$(x_i \vee \lnot{}x_j)$$        | $$x_i + (1-x_j) \geq 1$$     | $$(x_j - x_i x_j)$$
| Two Negations | $$(\lnot{}x_i \vee \lnot{}x_j)$$ | $$(1-x_i) + (1-x_j) \geq 1$$ | $$(x_i x_j)$$


Using these penalties we can transform our example into a QUBO model.

$$
y(\vec x) = 2-x_1-x_2-x_3-x_4+x_5+x_1*x_2+2*x_3*x_4-x_5*x_6
$$

$y(\vec x)$ is the integer of unsatisfied clauses. In other words, $y=0$ indicates that all clauses are satisfied, which is desired for this problem.

### Creating a cost function for feature costs

For the feature costs, we can formulate a sum that adds a features cost if it is in the input vector $\vec x$.
$$
k(\vec x) = \sum_{x}^{} c_i x_i
$$


### Combining the functions
In our example, we want to find a valid configuration with the minimum cost, so we sum the two previous functions and add a penalty factor $\alpha$ to be able to change the influence of our SAT constraints.

$$
f(\vec x) = k(\vec x) + \alpha y(\vec x)
$$

We assume $\alpha \gg 0$, the exact value probably depends on the value of $k(\vec x)$.

### Forming Hamiltonians
We got our QUBO cost function defined. We now need to transform the Binary input space $x_i \in \{0,1\}$ to the Ising Spin model of $z_i \in \{-1, 1\}$. 

Replace $x_i$ with $z_i = 2x_i-1$ meaning $x_i = \frac{1 - z_i}{2}$ (Note that switching the sign changes the eigenvectors)

#### Cost Hamiltonian for the SAT part
Before applying it to the entire function, let's first consider only $y(\vec x)$.

Ising Form:
$$
y(\vec z) = 2-\frac{1-z_1}{2}-\frac{1-z_2}{2}-\frac{1-z_3}{2}-\frac{1-z_4}{2}+\frac{1-z_5}{2}+\frac{1-z_1}{2}*\frac{1-z_2}{2}+2*\frac{1-z_3}{2}*\frac{1-z_4}{2}-\frac{1-z_5}{2}*\frac{1-z_6}{2}
$$
Simplified to (not really needed):
$$ 
y(\vec z) = (4 + z_2 + z_1 (1 + z_2) + 2 z_3 z_4 + z_6 - z_5 (1 + z_6))*\frac{1}{4}
$$
Which expands to:
$$
y(\vec z) = 1 + z_1\frac{1}{4} + z_2\frac{1}{4} + z_1 z_2\frac{1}{4} + z_3 z_4 \frac{1}{2} - z_5\frac{1}{4} + z_6\frac{1}{4} - z_5 z_6 \frac{1}{4}
$$
And leaves us with a cost hamiltonian $H_v$ with Pauli-Z-Gates $\sigma^z_i$ and an Identity $I$ on the global phase(?):
$$
H_v = 1*I + \sigma^z_1\frac{1}{4} + \sigma^z_2\frac{1}{4} + \sigma^z_1 \sigma^z_2\frac{1}{4} + \sigma^z_3 \sigma^z_4 \frac{1}{2} - \sigma^z_5\frac{1}{4} + \sigma^z_6\frac{1}{4} - \sigma^z_5 \sigma^z_6 \frac{1}{4}
$$

#### Cost Hamiltonian for the feature costs
We can expand the feature costs in our example according to the table above.
$$
k(\vec x) = \sum_{x}^{} c_i x_i = 30*x_1 + 20*x_2 + 25*x_3 + 50*x_4 + 10*x_5 + 10*x_6
$$
Which we then again transform into Ising form.

$$
k(\vec z) = 15*(1-z_1) + 10*(1-z_2)+ 12.5*(1-z_3) + 25*(1-z_4) + 5*(1-z_5) + 5*(1-z_6) \\
k(\vec z) = 72.5 - 15 z_1 - 10 z_2 - 12.5 z_3 - 25 z_4 - 5 z_5 - 5 z_6
$$

Which leaves us with our cost Hamiltonian $H_{\mathit{fc}}$
$$
H_{\mathit{fc}} = 72.5*I - 15 \sigma^z_1 - 10 \sigma^z_2 - 12.5 \sigma^z_3 - 25 \sigma^z_4 - 5 \sigma^z_5 - 5 \sigma^z_6
$$

#### Combining Hamiltonians

All that's left to do is choosing a suitable $\alpha$ and combining the functions.

We choose $\alpha = 200$.

$$
H_{C} = H_{\mathit{fc}} + \alpha ~ H_v
$$

$$
H_{C} = 72.5*I - 15 \sigma^z_1 - 10 \sigma^z_2 - 12.5 \sigma^z_3 - 25 \sigma^z_4 - 15 \sigma^z_5 - 5 \sigma^z_6  + 200*I + 50 \sigma^z_1 + 50 \sigma^z_2 + 50 \sigma^z_1 \sigma^z_2 + 100 \sigma^z_3 \sigma^z_4 - 50 \sigma^z_5 + 50 \sigma^z_6 - 50 \sigma^z_5 \sigma^z_6
$$

simplified to

$$
H_{C} = 272.5*I + 35 \sigma^z_1 + 40 \sigma^z_2 - 12.5 \sigma^z_3 -25 \sigma^z_4 - 55 \sigma^z_5 + 45 \sigma^z_6 + 50 \sigma^z_1 \sigma^z_2 + 100 \sigma^z_3 \sigma^z_4 - 50 \sigma^z_5 \sigma^z_6
$$

We can implement such a Hamiltonian $H_{C}$ using the `qubovert` library and solve small instances via bruteforce.

In [ ]:
from configproblem.util.hamiltonian_math import solve_bruteforce
%psource solve_bruteforce

In [ ]:
from qubovert import spin_var
# define spin variables 
z1, z2, z3, z4, z5, z6 = spin_var('z1'), spin_var('z2'), spin_var('z3'), spin_var('z4'), spin_var('z5'), spin_var('z6')

# Our manually calculated hamiltonian
feetcost_model = 272.5 + 35 * z1 + 40 * z2 - 12.5 * z3 - 25 * z4 - 55 * z5 + 45 * z6 + 50 * z1 * z2 + 100 * z3 * z4 - 50 * z5 * z6
solve_bruteforce(feetcost_model)

We can also define the SAT ($H_v$) and cost ($H_{\mathit{fc}}$) Hamiltonians separately and combine them afterwards.

In [ ]:
# cost and sat individually
from qubovert import boolean_var

# define binary vars
x1, x2, x3, x4, x5, x6 = boolean_var('x1'), boolean_var('x2'), boolean_var('x3'), boolean_var('x4'), boolean_var('x5'), boolean_var('x6')

# SAT Penalty
alpha_sat = 200 # 1e6

# SAT QUBO
sat_model = alpha_sat * (2 - x1 - x2 - x3 - x4 + x5 + x1 * x2 + 2 * x3 * x4 - x5 * x6)

# Cost QUBO
cost_model = 30*x1 + 20*x2 + 25*x3 + 50*x4 + 10*x5 + 10*x6

# Combine models
combined_model = sat_model +  cost_model
print("QUBO Combined Model:")
pprint(combined_model)
print("Ising Combined Model: ")
combined_hamiltonian = combined_model.to_quso()
print(combined_hamiltonian)

solve_bruteforce(combined_model)

## Classical Routine
On the classical side we now need functions to evaluate $C$, which correspond to computing the energy of the Hamiltonian for a specific measured output (a configuration in our case) by `compute_config_energy`.

In [ ]:
from configproblem.util.hamiltonian_math import compute_config_energy
%psource compute_config_energy

As the circuit is executed multiple times the function `compute_hamiltonian_energy` can be used with different strategies.
Currently, we provide average, top and minimum strategies and the best one to use is `'avg'`.

In [ ]:
from configproblem.util.hamiltonian_math import compute_hamiltonian_energy, hamiltonian_strategy_average, hamiltonian_strategy_top, hamiltonian_strategy_min
%psource compute_hamiltonian_energy
%psource hamiltonian_strategy_average
%psource hamiltonian_strategy_top
%psource hamiltonian_strategy_min

We can then define a function that given a beta and gamma as input creates a quantum circuit, executes it multiple times and returns the energy.

This function can be used in a classical optimizer.

In [ ]:
from qaoa_application import get_expectation
%psource get_expectation

In [ ]:
from configproblem.util.visualization import plot_beta_gamma_cost_landscape, plot_f_mu_cost_landscape
# Plot cost landscape for different values of beta and gamma
hamiltonians = [{'hamiltonian': sat_hamiltonian, 'name': 'SAT'},
                {'hamiltonian': cost_model.to_quso(), 'name': 'COST'},
                {'hamiltonian': combined_hamiltonian, 'name': 'COMBINED'}]
strategies = ['min', 'avg', 'top']
plot_beta_gamma_cost_landscape(problem_circuit, mixer.standard_mixer, hamiltonians, strategies, 6, 0.2)

# Plot cost landscape for different f and mu
plot_f_mu_cost_landscape(combined_hamiltonian, 6)

## Applying QAOA
Finally, we can run the whole algorithm for a specific problem Hamiltonian.

In [ ]:
from qaoa_application import apply_qaoa
%psource apply_qaoa

In [ ]:
import configproblem.qaoa.qaoa_parameter_optimization as parameter_optimization

# QAOA Example Application Using Optimizer
hamiltonian = combined_hamiltonian # min-cost SAT
# hamiltonian = sat_model.to_quso()   # just SAT

# warmstart array for specific SAT instance of this notebook
warmstart_statevector = \
      [0.        , 0.        , 0.        , 0.        , 0.        ,
       0.232379  , 0.28809721, 0.20976177, 0.        , 0.25298221,
       0.19493589, 0.24899799, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.20976177, 0.17888544, 0.23021729,
       0.        , 0.24899799, 0.23664319, 0.29664794, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.23021729, 0.24083189,
       0.26267851, 0.        , 0.232379  , 0.20248457, 0.20736441,
       0.04472136, 0.        , 0.        , 0.        ]

mixer_circuit = mixer.standard_mixer
parameter_optimization = parameter_optimization.get_optimizer('COBYLA')
layers = 5 # more layers = higher approximation rate but more quantum errors when running on real qpu
n_features = 6
shots = 256
theta = {"beta": 0.01, "gamma": -0.01} # start values for optimization
strategy = 'avg'
use_warmstart = False
use_optimizer = True

if not use_warmstart:
    warmstart_statevector = None

counts, qc = apply_qaoa(problem_circuit, mixer_circuit, parameter_optimization, hamiltonian, layers, n_features, shots, theta, warmstart_statevector, strategy=strategy, use_optimizer=use_optimizer)

In [ ]:
# qc.draw(output="mpl")

In [ ]:
# Pretty Print the results of the previous Cell
from configproblem.util.visualization import plot_counts_histogram

best_config = "000110" # 654321
valid_configs = ["101010", "101001", "101011", "100110", "100101", "100111", "001010", "001001", "001011", "000101", "000111", "111010", "111001", "111011", "110110", "110101", "110111"]

plot_counts_histogram(counts, n_features, best_config, valid_configs)

In [ ]:
# Visualize results using the StatevectorSimulator
from qaoa_application import apply_qaoa_statevector
import numpy as np

probabilities, _ = apply_qaoa_statevector(problem_circuit, mixer_circuit, parameter_optimization, hamiltonian, layers, n_features, theta, warmstart_statevector, strategy=strategy, use_optimizer=use_optimizer, print_res=False)
probabilities_dict = {}
for i in range(0, 2 ** n_features):
    probabilities_dict[(np.binary_repr(i, width=6))] = round(probabilities[i], 4)

plot_counts_histogram(probabilities_dict, n_features, best_config, valid_configs)

In [ ]:
# prioritization of features
from sympy.utilities.misc import ordinal
import operator

print_debug_output = True

# Config cost for each valid config
config_cost = {"000110":  45, "100110":  55, "000101":  55, "110110":  65, "100101":  65, "001010":  70,
               "110101":  75, "000111":  75, "001001":  80, "101010":  80, "111010":  90, "101001":  90,
               "100111":  95, "110111":  95, "001011": 100, "111001": 100, "101011": 110, "111011": 120}

# Sort counts by value -> prioritized list of configs
sorted_by_value_counts = dict(sorted(counts.items(), key=operator.itemgetter(1),reverse=True))

average_difference = 0
optimal_value_index = 0

# Evaluate prioritization results
for i, config in enumerate(sorted_by_value_counts):
    if config in config_cost:
        value = config_cost[config]
        optimal_value = list(config_cost.values())[optimal_value_index]
        optimal_value_index += 1
        counts_index, optimal_index = 0, 0

        config_cost_copy = config_cost.copy()
        while value in config_cost_copy.values():
            counts_index = list(config_cost_copy.values()).index(value)
            config_to_pop = list(config_cost_copy.keys())[list(config_cost_copy.values()).index(value)]
            config_cost_copy.pop(config_to_pop)

        config_cost_copy = config_cost.copy()
        while optimal_value in config_cost_copy.values():
            optimal_index = list(config_cost_copy.values()).index(optimal_value)
            config_to_pop = list(config_cost_copy.keys())[list(config_cost_copy.values()).index(optimal_value)]
            config_cost_copy.pop(config_to_pop)
        average_difference += abs(counts_index - optimal_index)

        if print_debug_output:
            print("Valid config " + str(config) + " with cost: " + str(config_cost[config]))
            print(ordinal(counts_index + 1) + " best config")
    else:
        if print_debug_output:
            print("Invalid config " + str(config))
print("Average difference: " + str(average_difference/len(config_cost)))